Multiscale optimization
===================

In [ ]:
import sys
import os
sys.path.append(os.path.realpath("../.."))
from scripts.preamble import *
recompute = True

In [ ]:
remesh_steps = pd.read_csv(os.path.join(basename, "remesh_steps.csv"), index_col=0).values[:,0]

In [ ]:
# Load wireframes
wireframes = []
renderings = []
vp = 6
collection = 25
res = 100
for i in range(len(remesh_steps)):
    filename = os.path.join(basename, f"res_{i:02d}_wireframe.png")
    if not os.path.exists(filename) or recompute:
        blender_render(os.path.join(basename, f"res_{i:02d}.ply"), 
                       basename, 
                       collection,
                       vp,
                       res=res,
                       ours=True,
                       wireframe=True,
                       t=0.006)
    wireframes.append(plt.imread(filename))

In [ ]:
fontsize = 36
sns.set_style('white')
base_size = 3
n_rows = 1
n_cols = len(wireframes)
h,w,_ = wireframes[0].shape
crops = np.zeros((len(wireframes), 2), dtype=np.int32)
for i, wf in enumerate(wireframes):
    nonzero = np.where(wf[..., 3] > 0)[1]
    crops[i] = (np.min(nonzero), np.max(nonzero))

widths = (crops[:,1] - crops[:,0]) / h
total_w = np.sum(widths)

fig = plt.figure(1, figsize=(total_w*base_size, n_rows * base_size), constrained_layout=True)
gs = fig.add_gridspec(n_rows,  n_cols, wspace=0.0,hspace=0.0, width_ratios=widths)

for j in range(n_cols):
    ax = fig.add_subplot(gs[j])

    im = ax.imshow(wireframes[j][:,crops[j,0]:crops[j,1]], interpolation='none')

    ax.tick_params(
        axis='both',
        which='both',
        labelleft=False,
        labelbottom=False)
    sns.despine(ax=ax, left=True, bottom=True)

arrow = matplotlib.patches.FancyArrowPatch(
    (0.05,0), (0.95,0), transform=fig.transFigure, fc='black', mutation_scale = 40.)
fig.patches.append(arrow)

fig.suptitle('Time', y=-0.05, fontsize=fontsize)
plt.savefig(os.path.join(basename, "multiscale.pdf"), format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.005)